In [1]:
import requests
from bs4 import BeautifulSoup
import pymongo
import time
import json
%matplotlib inline
import re

In [2]:
import json
f = open("movie.json", "r")
movie_data = json.load(f)
f.close()
movie_data = movie_data
print(movie_data[0].keys())
line = ""
for i in movie_data[0:100]:
    line += "{}\t".format(i['Mname'])
print(line)

[u'realease_kor', u'Webpage', u'Mname', u'Domestic', u'budget', u'gross_kor', u'release_usa', u'year', u'runtime', u'Oversea', u'rating_usa']
Toy Story 3	Alice in Wonderland (2010)	Harry Potter and the Deathly Hallows Part 1	Inception	Shrek Forever After	The Twilight Saga: Eclipse	Iron Man 2	Tangled	Despicable Me	How to Train Your Dragon	Clash of the Titans (2010)	The Chronicles of Narnia: The Voyage of the Dawn Treader	The King's Speech	Tron Legacy	The Karate Kid	Prince of Persia: The Sands of Time	Black Swan	Megamind	Robin Hood	The Last Airbender	Little Fockers	Resident Evil: Afterlife	Shutter Island	Salt	Sex and the City 2	The Tourist	The Expendables	Knight & Day	True Grit	Gulliver's Travels	Percy Jackson & The Olympians: The Lightning Thief	The Social Network	Valentine's Day	The Sorcerer's Apprentice	Due Date	Eat Pray Love	Red	Paranormal Activity 2	The A-Team	Jackass 3-D	Unstoppable	Step Up 3-D	The Book of Eli	The Town	Legend of the Guardians: The Owls of Ga'Hoole	The Wolfman	The B

In [3]:
class Database(object):
    def __init__(self):
        self.connection = pymongo.MongoClient("localhost", 27017)
        self.db = self.connection["DSS4_SmallP"]
        self.collection = self.db["Movie_mojo"]        
def Load_Data_mojo_yearly_dom(page,year):
    Mojo_url = u'http://www.boxofficemojo.com/'\
        u'yearly/chart/?page={}&view=releasedate&view2=domestic&yr={}&p=.htm'.format(page,year)
    return requests.get(Mojo_url)
def Load_Data_mojo_yearly_intl(year):
    Mojo_url = u'http://www.boxofficemojo.com/'\
        u'yearly/chart/?view2=worldwide&yr={}&p=.htm'.format(year)
    return requests.get(Mojo_url)
def Load_Data_mojo_movie_intl(M_id):
    Mojo_url = u'http://www.boxofficemojo.com{}&page=intl'.format(M_id)
    return requests.get(Mojo_url)
def Load_Data_mojo_movie(M_id):
    Mojo_url = u'http://www.boxofficemojo.com{}'.format(M_id)
    return requests.get(Mojo_url)

#Daum crawler
def load_apikeys():
    fp = open("private.json","r")
    res = json.load(fp)
    return res
def load_data_daum(query):
    key = load_apikeys()['daum_key']
    try :
        url = "https://apis.daum.net/contents/movie?apikey={}&q={}&output=json".format(key, query)
    except UnicodeEncodeError :
        query2 = re.match('[a-zA-Z\s]+',query)
        url = "https://apis.daum.net/contents/movie?apikey={}&q={}&output=json".format(key, query2.group())
    except Exception as err:
        print(err)
#    print(url)
    return requests.get(url)
def select_daum(query, movie):
    res = load_data_daum(query)
#    print(res)
    data = json.loads(res.content)
    #검색결과가 0개일때, 검색하는 제목에서 영어랑 띄어쓰기만 필터링해서 (ex. Godzilla (2014) -> Godzilla) 재검색
    if data['channel']['totalCount'] == 0:
        try : 
            query2 = re.match('[a-zA-Z\s]+',query)
            if query2.group() == query:
                return -2
            else:
                return select_daum(query2.group(), movie)
        except AttributeError:
            return -2
        except Exception as err:
            print(err)
            return -2
    #검색결과가 1개일때 결과값 출력
    elif data['channel']['totalCount'] == 1:
        return data['channel']["item"][0]
    #검색결과가 여러개일때, 개봉일을 비교하여 같은경우 출력, 개봉일이 같은게 없는경우 출력안함.
    else :
        mlist = []
        try:
            m = movie['realease_kor'].split('/')
            m = '{0:02d}{1:02d}{2:02d}'.format(int(m[2]),int(m[0]),int(m[1]))
            for item in data['channel']['item']:
                try:
                    d = item['open_info'][0]['content'].split('.')
                    d = '{0:02d}{1:02d}{2:02d}'.format(int(d[0][-2:]),int(d[1]),int(d[2]))
                    if d == m :
                        mlist.append(item)
                except:
                    pass
            if len(mlist) == 1:
                return mlist[0]
            else :
                if len(mlist) != 0 :print(len(mlist))
                return -1
        except Exception as err:
            print(err)
            return -1

In [10]:
print(movie_data[0].keys())

['m_name_k', 'grade', u'Oversea', u'gross_kor', 'daum_page', u'year', 'genre', 'viewing_time', u'realease_kor', 'director_k', u'Webpage', u'Mname', u'Domestic', u'budget', u'release_usa', 'actor_k', u'runtime', u'rating_usa']


In [11]:
arr_movie = []
err_movie = []
for arr_num, movie in enumerate(movie_data):
    res = select_daum(movie['Mname'],movie)
    dict_movie = {
        'm_name_k' : 'NA',
        'grade' : 'NA',
        'viewing_time' : -1,
        'genre' : 'NA',
        'director_k' : 'NA',
        'actor_k' : 'NA'
    }
    if res == -1 :
        err = "multiple : {}".format(movie['Mname'])
        err_movie.append(err)
    elif res == -2 : 
        err = "none : {}".format(res)
        err_movie.append(err)
    else :
        if res['open_info'][2]['content'] == '' :  res['open_info'][2]['content'] = '-1 '
        dict_movie = {
            'm_name_k' : res['title'][0]['content'],
            'grade' : res['open_info'][1]['content'],
            'viewing_time' : int(res['open_info'][2]['content'][0:-1]),
            'genre' : [X["content"] for X in res["genre"]],
            'director_k' : [(X["content"], X["link"]) for X in res["director"]],
            'actor_k' : [(X["content"], X["link"]) for X in res["actor"]],
            'daum_page' : res['title'][0]['link']
        }
    movie_data[arr_num].update(dict_movie)
fp = open("movie_added_daum.json", "w")
json.dump(movie_data, fp, indent = 4)
fp.close()

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range


In [12]:
#몽고DB
db = Database()
for movie in movie_data:
    db.collection.update_one({'Mname' : movie['Mname']}, {"$set":movie}, upsert=True)

In [39]:
import time
#영화 intl 페이지 -> 한국매출 / 개봉일 정보.
def load_data_mojo_movie_intl(M_id):
    Mojo_url = u'http://www.boxofficemojo.com{}&page=intl'.format(M_id)
    req = requests.get(Mojo_url)
    bsObj = BeautifulSoup(req.content,'html.parser')
    line_kor = bsObj.find('a', text = 'South Korea')
    for iter_kor in line_kor.parents:
        if re.match('<tr ',str(iter_kor)) :
            break
    gross_kor = iter_kor.find_all(text = re.compile('\$'))
    gross_kor = int(re.sub("[^0-9]","",gross_kor[-1]))
    release_kor = iter_kor.find_all(text = re.compile('\d+\/\d+/\d'))[0]
    return {
        'gross_kor' : gross_kor,
        'release_kor' : release_kor
    }
#영화 Main
def load_data_mojo_movie_main(M_id):
    Mojo_url = u'http://www.boxofficemojo.com{}&page=main'.format(M_id)
    req = requests.get(Mojo_url)
    bsObj = BeautifulSoup(req.content,'html.parser')
    #content = bsObj.find('td', attrs = {'valign' : 'top', 'align':'center'}).next_siblings
    tables = bsObj.find_all('table')
    for i in tables:
        if re.search(re.compile("Domestic Total Gross"),i.get_text())!=None:
            content = i
            break
    re_dir = re.compile('.+Director.+id=.+') #감독 추출용 표현식
    re_act = re.compile('.+Actor.+id=.+') #배우 추출용
    re_name = re.compile('[a-zA-Z\., ]+[a-zA-Z\.]') #이름 추출용(이름 외엔 지우는..)

    director = {}
    actor = {}
    franchise = {}

    #미국 매출 추출
    gross_us = str(content.find(text = 'Domestic Total Gross: ').next_sibling)
    gross_us = int(re.sub("[^0-9]","",gross_us))

    #제작비 추출
    project_budget = str(content.find(text = 'Production Budget: ').next_sibling.get_text())
    unit = project_budget.split(" ")[1]
    if unit == 'million' :
        unit = 1000000
    else :
        print("Need to add new unit : {}".format(unit))
    project_budget = int(re.sub("[^0-9]","",project_budget)) * unit

    #The Players부분 뽑아내기.
    people_links = bsObj.find_all('a', attrs = {"href" : re.compile('\/people\/[^"]')})

    #감독 추출
    for line in people_links:
        if re.search(re_dir, str(line)):
            break
    tag_directors = line.parent.find_all('a')
    #링크 있는 감독
    for iter_dir in tag_directors:
        name = re.match(re_name, iter_dir.get_text()).group()
        director[name] = iter_dir.attrs['href']
    #링크 없는 감독
    for split_dir in str(line.parent).split('<br>'):
        if not(re.search('<\/a>',split_dir)):
            name = re.match(re_name,split_dir).group()
            director[name] = 'NA'

    #배우 추출.
    for line in people_links:
        if re.search(re_act, str(line)):
            break
    tag_actors = line.parent.find_all('a')
    #링크 있는 배우들
    for iter_act in tag_actors:
        name = re.match(re_name, iter_act.get_text()).group()
        actor[name] = iter_act.attrs['href']
    #링크 없는 배우들
    for split_act in str(line.parent).split('<br>'):
        if not(re.search('<\/a>',split_act)):
            name = re.match(re_name,split_act).group()
            actor[name] = 'NA'

    #프랜차이즈
    divs = bsObj.find_all('div', attrs = {'class' : 'mp_box_tab'})
    for iter_div in divs :
        if re.search('.+Franchises.+',str(iter_div)):
            franchises_table = iter_div.parent
            break
    for iter_ser in franchises_table.find_all('a', text = re.compile('Series:')):
        franchise[iter_ser.get_text()] = iter_ser.attrs['href']
        
    return {
        'gross_us' : gross_us,
        'project_budget' : project_budget,
        'director' : director,
        'actor' : actor,
        'franchise' : franchise
    }

In [47]:
arr_movie = [{'mname' : 'test', 'm_id' : '/movies/?id=avengers2.htm'}]
# arr_movie = dict <- 영화정보 dict에 넣는 부분!
dict_web_actor = {}
dict_web_director = {}
dict_web_franchise = {}
dict_film_actor = {}
dict_film_director = {}
dict_film_franchise = {}

for N, iter_movie in enumerate(arr_movie):
    main = load_data_mojo_movie_main(iter_movie['m_id'])
    arr_movie[N].update(main)
    for k in main['actor'].keys():
        dict_web_actor[k] = main['actor'][k]
    for k in main['director'].keys():
        dict_web_director[k] = main['director'][k]
    for k in main['franchise'].keys():
        dict_web_franchise[k] = main['franchise'][k]
    arr_movie[N].update(load_data_mojo_movie_intl(iter_movie['m_id']))

In [49]:
dict_web_actor

{u'Aaron Johnson': u'/people/chart/?view=Actor&id=aaronjohnson.htm',
 u'Andy Serkis': u'/people/chart/?view=Actor&id=andyserkis.htm',
 u'Anthony Mackie': u'/people/chart/?view=Actor&id=anthonymackie.htm',
 u'Chris Evans': u'/people/chart/?view=Actor&id=chrisevans.htm',
 u'Chris Hemsworth': u'/people/chart/?view=Actor&id=chrishemsworth.htm',
 'Cobie Smulders': 'NA',
 u'Don Cheadle': u'/people/chart/?view=Actor&id=doncheadle.htm',
 u'Elizabeth Olsen': u'/people/chart/?view=Actor&id=elizabetholsen.htm',
 'Hayley Atwell': 'NA',
 u'Idris Elba': u'/people/chart/?view=Actor&id=idriselba.htm',
 'James Spader': 'NA',
 u'Jeremy Renner': u'/people/chart/?view=Actor&id=jeremyrenner.htm',
 u'Josh Brolin': u'/people/chart/?view=Actor&id=joshbrolin.htm',
 u'Mark Ruffalo': u'/people/chart/?view=Actor&id=markruffalo.htm',
 u'Paul Bettany': u'/people/chart/?view=Actor&id=paulbettany.htm',
 u'Robert Downey, Jr.': u'/people/chart/?view=Actor&id=robertdowneyjr.htm',
 u'Samuel L. Jackson': u'/people/chart/?